In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd ./drive/MyDrive/yolov7

Mounted at /content/drive
/content/drive/MyDrive/yolov7


In [ ]:
!pip install pysrt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pysrt: filename=pysrt-1.1.2-py3-none-any.whl size=13443 sha256=038ea67c814f336a62c7e4f3841eeafc98436bbf0e2b191d343d5b703f276302
  Stored in directory: /root/.cache/pip/wheels/30/7f/e8/55de9a9b07302d9e7fe47c27910e3bea0c48536153e74bd7e6
Successfully built pysrt


In [ ]:
import csv
import pysrt

def srt_to_csv(srt_file, csv_file):
    try:
        # SRTファイルを読み込み
        subs = pysrt.open(srt_file, encoding='utf-8')

        # CSVファイルに書き込み
        with open(csv_file, 'w', newline='', encoding='utf-8') as csvfile:
            csv_writer = csv.writer(csvfile)
            # CSVヘッダー
            csv_writer.writerow(['Index', 'Start Time', 'End Time', 'Text'])

            # 各セグメントをCSV行として書き込み
            for sub in subs:
                csv_writer.writerow([sub.index, sub.start, sub.end, sub.text])

        print(f"'{srt_file}'を'{csv_file}'に変換しました。")

    except Exception as e:
        print(f"エラーが発生しました: {e}")

# 使用例
srt_file = 'ex/timetable/srt/nabe_vrew.srt'  # SRTファイルのパスを指定
csv_file = 'ex/timetable/csv/nabe.csv'  # 出力先CSVファイルのパスを指定
srt_to_csv(srt_file, csv_file)


'ex/timetable/srt/nabe_vrew.srt'を'ex/timetable/csv/nabe.csv'に変換しました。


# 新しいセクション

# 新しいセクション

In [ ]:
import csv

# 時刻を秒数に変換する関数
def time_to_seconds(time_str):
    """時刻（HH:MM:SS,ms形式）のミリ秒を無視して秒に変換する関数"""
    time_str = time_str.strip().split(',')[0]  # カンマ以降を無視して、空白とBOMを除去
    try:
        h, m, s = map(int, time_str.split(':'))
        return h * 3600 + m * 60 + s
    except ValueError:
        print(f"Invalid time format: '{time_str}'. Expected format: HH:MM:SS.")
        return None

# 行動ラベルを追加する関数
def add_action_labels(keypoints_file, timetable_file, output_file):
    # タイムテーブルを読み込んでラベルと時間帯をマッピング
    actions = []
    with open(timetable_file, 'r', encoding='utf-8-sig') as file:
        reader = csv.reader(file)
        next(reader)  # ヘッダー行をスキップ
        for row in reader:
            if len(row) < 4:
                continue
            start_time, end_time, label = row[1], row[2], row[3]
            start_sec = time_to_seconds(start_time)
            end_sec = time_to_seconds(end_time)
            if start_sec is not None and end_sec is not None:
                actions.append((start_sec, end_sec, label))

    # キーポイントデータに行動ラベルを追加
    with open(keypoints_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', newline='', encoding='utf-8') as outfile:
        reader = csv.reader(infile)
        writer = csv.writer(outfile)

        # ヘッダー処理
        header = next(reader)
        header.append("Action Label")
        writer.writerow(header)

        # 各フレームに対応するラベルを確認
        fps = 30  # フレームレート
        for row in reader:
            frame_id = int(row[0])
            time_sec = frame_id // fps
            action_label = "None"  # デフォルトのラベル

            # 該当するラベルを確認
            for start_sec, end_sec, label in actions:
                if start_sec <= time_sec <= end_sec:
                    action_label = label
                    break

            # 行動ラベルを追加して書き込み
            row.append(action_label)
            writer.writerow(row)




In [ ]:
keypoints_file = 'ex/keypoint/video_nabe.csv'     # キーポイントデータのCSVファイル
timetable_file = 'ex/timetable/csv/nabe.csv'     # タイムテーブル（行動ラベル付き）のCSVファイル
output_file = 'ex/keylabel/keypoints_with_labels_nabe.csv'  # 行動ラベル付きで出力するCSVファイル

add_action_labels(keypoints_file, timetable_file, output_file)

In [ ]:
import csv

def count_action_occurrences(csv_file, action_label):
    count = 0
    with open(csv_file, 'r', encoding='utf-8') as file:
        reader = csv.DictReader(file)
        for row in reader:
            if row['Action Label'] == action_label:
                count += 1
    return count

# 使用例
csv_file = 'ex/keypoints_with_labels_right.csv'
action_label = 'Attacking'
count = count_action_occurrences(csv_file, action_label)
print(f"'{action_label}' の出現回数: {count}")


'Attacking' の出現回数: 4350


０の行を表示

In [ ]:
import os
import csv
from collections import Counter

# 入力ファイルのリスト
file_paths = {
    'person1': 'data/feature/new_feature/features_with_labels_prerna.csv',
    'person2': 'data/feature/new_feature/features_with_labels_umang.csv',
    'person3': 'data/feature/new_feature/features_with_labels_yutaro.csv',
    'person4': 'data/feature/new_feature/features_with_labels_aoi.csv',
    'person5': 'data/feature/new_feature/features_with_labels_nabe.csv'
}

# 全てのファイルのアクションカウントをまとめる
total_action_counts = Counter()

# 各ファイルの処理
for input_file in input_files:
    if not os.path.exists(input_file):
        print(f"Skipping {input_file}: File not found.")
        continue

    print(f"Processing {input_file}...")

    with open(input_file, 'r') as infile:
        reader = csv.DictReader(infile)
        action_counts = Counter(row['Action Label'].strip() for row in reader if row['Action Label'].strip())

    # ファイルごとのカウントを表示
    print(f"Action counts in {input_file}:")
    for action, count in action_counts.items():
        print(f"  {action}: {count}")

    # 全体のカウントに加算
    total_action_counts.update(action_counts)

# 全ファイルを統合した結果を表示
print("\nTotal Action Counts Across All Files:")
for action, count in total_action_counts.items():
    print(f"  {action}: {count}")


Processing Experiment/data/processed/keypointlabel/keypoints_with_labels_nabe.csv...
Action counts in Experiment/data/processed/keypointlabel/keypoints_with_labels_nabe.csv:
  Walking: 14460
  Head banging: 3930
  None: 3388
  Sitting quietly: 14700
  Biting: 2790
  Using phone: 10980
  Throwing things: 3960
  Eating snacks: 14670
  Attacking: 5760
Processing Experiment/data/processed/keypointlabel/keypoints_with_labels_prerna.csv...
Action counts in Experiment/data/processed/keypointlabel/keypoints_with_labels_prerna.csv:
  Sitting quietly: 27450
  Using phone: 24120
  Head banging: 6210
  None: 11617
  Walking: 20100
  Eating snacks: 10620
  Throwing things: 3750
  Attacking: 5340
  Biting: 8880
Processing Experiment/data/processed/keypointlabel/keypoints_with_labels_umang.csv...
Action counts in Experiment/data/processed/keypointlabel/keypoints_with_labels_umang.csv:
  Sitting quietly: 24000
  None: 9081
  Using phone: 23430
  Head banging: 5880
  Walking: 20190
  Eating snacks: 188

In [15]:
import os
import csv
import math
from collections import deque

# 異常行動のウィンドウサイズとオーバーラップ
abnormal_window_params = {
    "Attacking": (20, 10),
    "Biting": (20, 10),
    "Head banging": (20, 15),
    "Throwing things": (25, 20),
}

# 正常行動のウィンドウサイズとオーバーラップ
normal_window_params = {
    "Walking": (90, 45),
    "Sitting quietly": (90, 45),
    "Using phone": (90, 45),
    "Eating snacks": (90, 45),
}

# 特徴量計算関数
def calculate_speed(positions):
    if len(positions) < 2:
        return 0
    return math.sqrt((positions[-1][0] - positions[0][0]) ** 2 + (positions[-1][1] - positions[0][1]) ** 2)

def calculate_acceleration(positions):
    if len(positions) < 3:
        return 0
    v1 = math.sqrt((positions[-2][0] - positions[-3][0]) ** 2 + (positions[-2][1] - positions[-3][1]) ** 2)
    v2 = math.sqrt((positions[-1][0] - positions[-2][0]) ** 2 + (positions[-1][1] - positions[-2][1]) ** 2)
    return abs(v2 - v1)

def calculate_angle(p1, p2, p3):
    try:
        angle = math.degrees(math.atan2(p3[1] - p1[1], p3[0] - p1[0]) - math.atan2(p2[1] - p1[1], p2[0] - p1[0]))
        return abs(angle)
    except:
        return 0

def calculate_vertical_movement(positions):
    if len(positions) < 2:
        return 0
    return abs(positions[-1][1] - positions[0][1])

def calculate_horizontal_movement(positions):
    if len(positions) < 2:
        return 0
    return abs(positions[-1][0] - positions[0][0])

def calculate_distance(p1, p2):
    try:
        return math.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2)
    except:
        return 0

def generate_windows(data, window_size, overlap):
    step = window_size - overlap
    windows = []
    for i in range(0, len(data) - window_size + 1, step):
        windows.append(data[i:i + window_size])
    return windows

# 特徴量を計算しCSVに保存する関数
def calculate_features_and_save(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
        reader = csv.DictReader(infile)
        data = list(reader)
        if not data:
            print(f"Warning: {input_file} is empty.")
            return

        fieldnames = [
            'action', 'right_hand_speed', 'left_hand_speed', 'right_hand_acceleration', 'left_hand_acceleration',
            'right_shoulder_wrist_angle', 'left_shoulder_wrist_angle', 'right_eye_vertical_movement', 'left_eye_vertical_movement',
            'right_eye_horizontal_movement', 'left_eye_horizontal_movement', 'right_hand_eye_distance', 'left_hand_eye_distance',
            'right_foot_speed', 'left_foot_speed'
        ]
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        action_labels = [row['Action Label'] for row in data]
        unique_actions = set(action_labels)

        for action in unique_actions:
            window_size, overlap = abnormal_window_params.get(action, normal_window_params.get(action, (60, 50)))
            action_data = [row for row in data if row['Action Label'] == action]
            windows = generate_windows(action_data, window_size, overlap)

            for window in windows:
                if not window:
                    continue

                right_hand_positions = deque(maxlen=window_size)
                left_hand_positions = deque(maxlen=window_size)
                right_eye_positions = deque(maxlen=window_size)
                left_eye_positions = deque(maxlen=window_size)

                for row in window:
                    right_hand_positions.append((float(row.get('right_wrist_x', 0)), float(row.get('right_wrist_y', 0))))
                    left_hand_positions.append((float(row.get('left_wrist_x', 0)), float(row.get('left_wrist_y', 0))))
                    right_eye_positions.append((float(row.get('right_eye_x', 0)), float(row.get('right_eye_y', 0))))
                    left_eye_positions.append((float(row.get('left_eye_x', 0)), float(row.get('left_eye_y', 0))))

                features = {
                    'action': window[-1]['Action Label'],
                    'right_hand_speed': calculate_speed(right_hand_positions),
                    'left_hand_speed': calculate_speed(left_hand_positions),
                    'right_hand_acceleration': calculate_acceleration(right_hand_positions),
                    'left_hand_acceleration': calculate_acceleration(left_hand_positions),
                    'right_shoulder_wrist_angle': calculate_angle(right_hand_positions[0], right_eye_positions[0], right_hand_positions[-1]),
                    'left_shoulder_wrist_angle': calculate_angle(left_hand_positions[0], left_eye_positions[0], left_hand_positions[-1]),
                    'right_eye_vertical_movement': calculate_vertical_movement(right_eye_positions),
                    'left_eye_vertical_movement': calculate_vertical_movement(left_eye_positions),
                    'right_eye_horizontal_movement': calculate_horizontal_movement(right_eye_positions),
                    'left_eye_horizontal_movement': calculate_horizontal_movement(left_eye_positions),
                    'right_hand_eye_distance': calculate_distance(right_hand_positions[0], right_eye_positions[0]),
                    'left_hand_eye_distance': calculate_distance(left_hand_positions[0], left_eye_positions[0]),
                    'right_foot_speed': calculate_speed(right_hand_positions),
                    'left_foot_speed': calculate_speed(left_hand_positions),
                }
                writer.writerow(features)

# データ処理
input_files = [
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_nabe.csv',
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_prerna.csv',
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_umang.csv',
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_yutaro.csv',
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_aoi.csv'
]

output_dir = 'Experiment/data/feature/new_feature'
os.makedirs(output_dir, exist_ok=True)

for input_file in input_files:
    output_file = os.path.join(output_dir, os.path.basename(input_file).replace('keypoints', 'features'))
    calculate_features_and_save(input_file, output_file)
    print(f"Processed and saved: {output_file}")


Processed and saved: Experiment/data/feature/new_feature/features_with_labels_nabe.csv
Processed and saved: Experiment/data/feature/new_feature/features_with_labels_prerna.csv
Processed and saved: Experiment/data/feature/new_feature/features_with_labels_umang.csv
Processed and saved: Experiment/data/feature/new_feature/features_with_labels_yutaro.csv
Processed and saved: Experiment/data/feature/new_feature/features_with_labels_aoi.csv


特徴量計算

In [ ]:
import os
import csv
import math

def calculate_speed(positions):
    if len(positions) < 2:
        return 0
    return math.sqrt((positions[-1][0] - positions[0][0]) ** 2 + (positions[-1][1] - positions[0][1]) ** 2)

def calculate_acceleration(positions):
    if len(positions) < 3:
        return 0
    v1 = math.sqrt((positions[-2][0] - positions[-3][0]) ** 2 + (positions[-2][1] - positions[-3][1]) ** 2)
    v2 = math.sqrt((positions[-1][0] - positions[-2][0]) ** 2 + (positions[-1][1] - positions[-2][1]) ** 2)
    return abs(v2 - v1)

def calculate_angle(p1, p2, p3):
    try:
        angle = math.degrees(math.atan2(p3[1] - p1[1], p3[0] - p1[0]) - math.atan2(p2[1] - p1[1], p2[0] - p1[0]))
        return abs(angle)
    except:
        return 0

def calculate_vertical_movement(p1, p2):
    return abs(p2[1] - p1[1])

def calculate_horizontal_movement(p1, p2):
    return abs(p2[0] - p1[0])

def calculate_distance(p1, p2):
    try:
        return math.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2)
    except:
        return 0

def calculate_features_and_save(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
        reader = csv.DictReader(infile)
        fieldnames = [
            'action',
            'right_hand_speed', 'left_hand_speed',
            'right_hand_acceleration', 'left_hand_acceleration',
            'right_shoulder_wrist_angle', 'left_shoulder_wrist_angle',
            'right_eye_vertical_movement', 'left_eye_vertical_movement',
            'right_eye_horizontal_movement', 'left_eye_horizontal_movement',
            'right_hand_eye_distance', 'left_hand_eye_distance',
            'right_foot_speed', 'left_foot_speed',
        ]
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        previous_positions = {
            'right_hand': None, 'left_hand': None,
            'right_foot': None, 'left_foot': None,
            'right_eye': None, 'left_eye': None
        }

        for row in reader:
            # 現在の位置を取得
            right_hand = (float(row.get('right_wrist_x', 0)), float(row.get('right_wrist_y', 0)))
            left_hand = (float(row.get('left_wrist_x', 0)), float(row.get('left_wrist_y', 0)))
            right_foot = (float(row.get('right_ankle_x', 0)), float(row.get('right_ankle_y', 0)))
            left_foot = (float(row.get('left_ankle_x', 0)), float(row.get('left_ankle_y', 0)))
            right_eye = (float(row.get('right_eye_x', 0)), float(row.get('right_eye_y', 0)))
            left_eye = (float(row.get('left_eye_x', 0)), float(row.get('left_eye_y', 0)))
            right_shoulder = (float(row.get('right_shoulder_x', 0)), float(row.get('right_shoulder_y', 0)))
            left_shoulder = (float(row.get('left_shoulder_x', 0)), float(row.get('left_shoulder_y', 0)))

            # 特徴量計算
            features = {
                'action': row['Action Label'],
                'right_hand_speed': calculate_speed([previous_positions['right_hand'], right_hand]) if previous_positions['right_hand'] else 0,
                'left_hand_speed': calculate_speed([previous_positions['left_hand'], left_hand]) if previous_positions['left_hand'] else 0,
                'right_hand_acceleration': calculate_acceleration([previous_positions['right_hand'], right_hand]) if previous_positions['right_hand'] else 0,
                'left_hand_acceleration': calculate_acceleration([previous_positions['left_hand'], left_hand]) if previous_positions['left_hand'] else 0,
                'right_shoulder_wrist_angle': calculate_angle(right_shoulder, previous_positions['right_hand'], right_hand) if previous_positions['right_hand'] else 0,
                'left_shoulder_wrist_angle': calculate_angle(left_shoulder, previous_positions['left_hand'], left_hand) if previous_positions['left_hand'] else 0,
                'right_eye_vertical_movement': calculate_vertical_movement(previous_positions['right_eye'], right_eye) if previous_positions['right_eye'] else 0,
                'left_eye_vertical_movement': calculate_vertical_movement(previous_positions['left_eye'], left_eye) if previous_positions['left_eye'] else 0,
                'right_eye_horizontal_movement': calculate_horizontal_movement(previous_positions['right_eye'], right_eye) if previous_positions['right_eye'] else 0,
                'left_eye_horizontal_movement': calculate_horizontal_movement(previous_positions['left_eye'], left_eye) if previous_positions['left_eye'] else 0,
                'right_hand_eye_distance': calculate_distance(right_hand, right_eye),
                'left_hand_eye_distance': calculate_distance(left_hand, left_eye),
                'right_foot_speed': calculate_speed([previous_positions['right_foot'], right_foot]) if previous_positions['right_foot'] else 0,
                'left_foot_speed': calculate_speed([previous_positions['left_foot'], left_foot]) if previous_positions['left_foot'] else 0,
            }

            writer.writerow(features)

            # 現在の位置を保存
            previous_positions['right_hand'] = right_hand
            previous_positions['left_hand'] = left_hand
            previous_positions['right_foot'] = right_foot
            previous_positions['left_foot'] = left_foot
            previous_positions['right_eye'] = right_eye
            previous_positions['left_eye'] = left_eye

# 複数ファイルの処理
input_files = [
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_nabe.csv',
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_prerna.csv',
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_umang.csv',
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_yutaro.csv',
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_aoi.csv'
]

output_dir = 'Experiment/data/feature/new_feature'
os.makedirs(output_dir, exist_ok=True)

for input_file in input_files:
    output_file = os.path.join(output_dir, os.path.basename(input_file).replace('keypoints', 'features'))
    calculate_features_and_save(input_file, output_file)
    print(f"Processed and saved: {output_file}")


Processed and saved: Experiment/data/feature/new_feature/features_with_labels_nabe.csv
Processed and saved: Experiment/data/feature/new_feature/features_with_labels_prerna.csv
Processed and saved: Experiment/data/feature/new_feature/features_with_labels_umang.csv
Processed and saved: Experiment/data/feature/new_feature/features_with_labels_yutaro.csv
Processed and saved: Experiment/data/feature/new_feature/features_with_labels_aoi.csv


In [16]:
import os
import csv
import math
from collections import deque

# 特徴量計算の関数（変更なし）
def calculate_speed(positions):
    if len(positions) < 2:
        return 0
    return math.sqrt((positions[-1][0] - positions[0][0]) ** 2 + (positions[-1][1] - positions[0][1]) ** 2)

def calculate_acceleration(positions):
    if len(positions) < 3:
        return 0
    v1 = math.sqrt((positions[-2][0] - positions[-3][0]) ** 2 + (positions[-2][1] - positions[-3][1]) ** 2)
    v2 = math.sqrt((positions[-1][0] - positions[-2][0]) ** 2 + (positions[-1][1] - positions[-2][1]) ** 2)
    return abs(v2 - v1)

def calculate_angle(p1, p2, p3):
    try:
        angle = math.degrees(math.atan2(p3[1] - p1[1], p3[0] - p1[0]) - math.atan2(p2[1] - p1[1], p2[0] - p1[0]))
        return abs(angle)
    except:
        return 0

def calculate_vertical_movement(positions):
    if len(positions) < 2:
        return 0
    return abs(positions[-1][1] - positions[0][1])

def calculate_horizontal_movement(positions):
    if len(positions) < 2:
        return 0
    return abs(positions[-1][0] - positions[0][0])

def calculate_distance(p1, p2):
    try:
        return math.sqrt((p2[0] - p1[0]) ** 2 + (p2[1] - p1[1]) ** 2)
    except:
        return 0

def calculate_distance_change(distances):
    if len(distances) < 2:
        return 0
    return abs(distances[-1] - distances[0])

# ウィンドウを生成する関数
def generate_windows(data, window_size, overlap):
    step = window_size - overlap
    windows = []
    for i in range(0, len(data) - window_size + 1, step):
        windows.append(data[i:i + window_size])
    return windows

# 特徴量を計算しCSVに保存する関数
def calculate_features_and_save(input_file, output_file, window_size=30, overlap=15):
    with open(input_file, 'r') as infile, open(output_file, 'w', newline='') as outfile:
        reader = csv.DictReader(infile)
        data = list(reader)  # 全データをリストに変換
        fieldnames = [
            'action',
            'right_hand_speed', 'left_hand_speed',
            'right_hand_acceleration', 'left_hand_acceleration',
            'right_shoulder_wrist_angle', 'left_shoulder_wrist_angle',
            'right_eye_vertical_movement', 'left_eye_vertical_movement',
            'right_eye_horizontal_movement', 'left_eye_horizontal_movement',
            'right_hand_eye_distance', 'left_hand_eye_distance',
            'right_foot_speed', 'left_foot_speed',
        ]
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        # ウィンドウを生成
        windows = generate_windows(data, window_size, overlap)

        # 各ウィンドウで特徴量を計算
        for window in windows:
            # キーポイントデータを蓄積するためのキュー
            right_hand_positions = deque(maxlen=window_size)
            left_hand_positions = deque(maxlen=window_size)
            right_foot_positions = deque(maxlen=window_size)
            left_foot_positions = deque(maxlen=window_size)
            right_eye_positions = deque(maxlen=window_size)
            left_eye_positions = deque(maxlen=window_size)
            right_distances = deque(maxlen=window_size)
            left_distances = deque(maxlen=window_size)

            for row in window:
                right_hand_positions.append((float(row.get('right_wrist_x', 0)), float(row.get('right_wrist_y', 0))))
                left_hand_positions.append((float(row.get('left_wrist_x', 0)), float(row.get('left_wrist_y', 0))))
                right_foot_positions.append((float(row.get('right_ankle_x', 0)), float(row.get('right_ankle_y', 0))))
                left_foot_positions.append((float(row.get('left_ankle_x', 0)), float(row.get('left_ankle_y', 0))))
                right_eye_positions.append((float(row.get('right_eye_x', 0)), float(row.get('right_eye_y', 0))))
                left_eye_positions.append((float(row.get('left_eye_x', 0)), float(row.get('left_eye_y', 0))))
                right_shoulder = (float(row.get('right_shoulder_x', 0)), float(row.get('right_shoulder_y', 0)))
                left_shoulder = (float(row.get('left_shoulder_x', 0)), float(row.get('left_shoulder_y', 0)))
                right_eye = right_eye_positions[-1]
                left_eye = left_eye_positions[-1]

                right_distances.append(calculate_distance(right_hand_positions[-1], right_eye))
                left_distances.append(calculate_distance(left_hand_positions[-1], left_eye))

            # 特徴量の計算
            features = {
                'action': window[-1]['Action Label'],  # 最後のフレームのラベルを使用
                'right_hand_speed': calculate_speed(right_hand_positions),
                'left_hand_speed': calculate_speed(left_hand_positions),
                'right_hand_acceleration': calculate_acceleration(right_hand_positions),
                'left_hand_acceleration': calculate_acceleration(left_hand_positions),
                'right_shoulder_wrist_angle': calculate_angle(right_shoulder, right_hand_positions[0], right_hand_positions[-1]),
                'left_shoulder_wrist_angle': calculate_angle(left_shoulder, left_hand_positions[0], left_hand_positions[-1]),
                'right_eye_vertical_movement': calculate_vertical_movement(right_eye_positions),
                'left_eye_vertical_movement': calculate_vertical_movement(left_eye_positions),
                'right_eye_horizontal_movement': calculate_horizontal_movement(right_eye_positions),
                'left_eye_horizontal_movement': calculate_horizontal_movement(left_eye_positions),
                'right_hand_eye_distance': calculate_distance(right_hand_positions[0], right_eye),
                'left_hand_eye_distance': calculate_distance(left_hand_positions[0], left_eye),
                'right_foot_speed': calculate_speed(right_foot_positions),
                'left_foot_speed': calculate_speed(left_foot_positions),
            }

            writer.writerow(features)

# 複数ファイルの処理
input_files = [
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_nabe.csv',
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_prerna.csv',
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_umang.csv',
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_yutaro.csv',
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_aoi.csv'
]

output_dir = 'Experiment/data/feature/new_feature'
os.makedirs(output_dir, exist_ok=True)

for input_file in input_files:
    output_file = os.path.join(output_dir, os.path.basename(input_file).replace('keypoints', 'features'))
    calculate_features_and_save(input_file, output_file)
    print(f"Processed and saved: {output_file}")


Processed and saved: Experiment/data/feature/new_feature/features_with_labels_nabe.csv
Processed and saved: Experiment/data/feature/new_feature/features_with_labels_prerna.csv
Processed and saved: Experiment/data/feature/new_feature/features_with_labels_umang.csv
Processed and saved: Experiment/data/feature/new_feature/features_with_labels_yutaro.csv
Processed and saved: Experiment/data/feature/new_feature/features_with_labels_aoi.csv


クロスバリデーション

In [ ]:
pwd

'/content/drive/My Drive/yolov7'

In [ ]:
import pandas as pd

# 全てのデータセットを確認
input_files = [
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_nabe.csv',
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_prerna.csv',
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_umang.csv',
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_yutaro.csv',
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_aoi.csv'
]

# 各ファイルで150フレーム以上の連続 'Head banging' を確認（フレームIDを考慮）
for file_path in input_files:
    df = pd.read_csv(file_path)

    max_length = 0
    current_length = 0
    previous_frame = None

    for index, row in df.iterrows():
        action = row['Action Label']
        frame_id = row['frame_id']

        if action == 'Head banging':
            # 連続したフレームかを確認
            if previous_frame is None or frame_id == previous_frame + 1:
                current_length += 1
            else:
                max_length = max(max_length, current_length)
                current_length = 1
            previous_frame = frame_id
        else:
            max_length = max(max_length, current_length)
            current_length = 0
            previous_frame = None

    # 最終的な最大連続長を更新
    max_length = max(max_length, current_length)

    print(f"{file_path}: Head bangingの最大連続フレーム長: {max_length}")

    if max_length < 150:
        print("⚠️ このデータには150フレームのHead bangingシーケンスが存在しません！")
    else:
        print("✅ 150フレーム以上の連続シーケンスが存在します！")


Experiment/data/processed/keypointlabel/keypoints_with_labels_nabe.csv: Head bangingの最大連続フレーム長: 2100
✅ 150フレーム以上の連続シーケンスが存在します！
Experiment/data/processed/keypointlabel/keypoints_with_labels_prerna.csv: Head bangingの最大連続フレーム長: 2010
✅ 150フレーム以上の連続シーケンスが存在します！
Experiment/data/processed/keypointlabel/keypoints_with_labels_umang.csv: Head bangingの最大連続フレーム長: 1920
✅ 150フレーム以上の連続シーケンスが存在します！
Experiment/data/processed/keypointlabel/keypoints_with_labels_yutaro.csv: Head bangingの最大連続フレーム長: 2160
✅ 150フレーム以上の連続シーケンスが存在します！
Experiment/data/processed/keypointlabel/keypoints_with_labels_aoi.csv: Head bangingの最大連続フレーム長: 2010
✅ 150フレーム以上の連続シーケンスが存在します！


In [ ]:
import pandas as pd
import numpy as np

file_paths = [
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_nabe.csv',
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_prerna.csv',
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_umang.csv',
    'Experiment/data/processed/keypointlabel/keypoints_with_labels_yutaro.csv',
]

time_steps = 150
target_action = 'Head banging'

def count_sequences_for_action(file_paths, target_action, time_steps):
    total_sequences = 0

    for file_path in file_paths:
        df = pd.read_csv(file_path)
        X = df.drop('Action Label', axis=1)
        y = df['Action Label']

        buffer = []
        last_action = None
        action_sequences = 0

        for i in range(len(y)):
            action = y.iloc[i]

            if action == target_action:
                if action == last_action or last_action is None:
                    buffer.append(X.iloc[i].values)
                else:
                    buffer = [X.iloc[i].values]

                if len(buffer) == time_steps:
                    action_sequences += 1
                    buffer.pop(0)
            else:
                buffer = []

            last_action = action

        print(f'{file_path}: {action_sequences} sequences of "{target_action}"')
        total_sequences += action_sequences

    print(f'Total sequences of "{target_action}" across all files: {total_sequences}')

count_sequences_for_action(file_paths, target_action, time_steps)


Experiment/data/processed/keypointlabel/keypoints_with_labels_nabe.csv: 3632 sequences of "Head banging"
Experiment/data/processed/keypointlabel/keypoints_with_labels_prerna.csv: 5614 sequences of "Head banging"
Experiment/data/processed/keypointlabel/keypoints_with_labels_umang.csv: 5284 sequences of "Head banging"
Experiment/data/processed/keypointlabel/keypoints_with_labels_yutaro.csv: 3632 sequences of "Head banging"
Total sequences of "Head banging" across all files: 18162


In [ ]:
%cd Experiment
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder


def debug_head_banging_sequences(file_paths, time_steps=150):
    label_encoder = None
    all_data = pd.concat([pd.read_csv(f) for f in file_paths.values()], ignore_index=True)

    # ラベルエンコーダの確認
    if 'action' not in all_data.columns:
        raise ValueError("Column 'action' not found in dataset")

    label_encoder = LabelEncoder()
    label_encoder.fit(all_data['action'])
    print("LabelEncoder classes:", label_encoder.classes_)

    target_label = 'Head banging'
    if target_label not in label_encoder.classes_:
        raise ValueError(f"Label '{target_label}' not found in classes")

    target_encoded = label_encoder.transform([target_label])[0]

    for file_name, file_path in file_paths.items():
        df = pd.read_csv(file_path)
        if 'action' not in df.columns:
            raise ValueError(f"Column 'action' not found in file {file_name}")

        X = df.drop(columns=['action'])
        y = df['action']

        # デバッグ用カウント変数
        head_banging_count = 0
        sequences = []
        buffer = []
        last_action = None

        y_encoded = label_encoder.transform(y)

        for i in range(len(X)):
            action = y_encoded[i]

            # バッファに追加
            if action == last_action or last_action is None:
                buffer.append(X.iloc[i].values)
            else:
                buffer = [X.iloc[i].values]

            # Head banging カウント
            if action == target_encoded:
                head_banging_count += 1

            # 150フレームに達したらシーケンスを作成
            if len(buffer) == time_steps:
                sequences.append(np.array(buffer))
                buffer.pop(0)

            last_action = action

        print(f"File: {file_name}")
        print(f"Number of 'Head banging' frames: {head_banging_count}")
        print(f"Number of sequences created: {len(sequences)}")

    print("All checks completed.")


# 使用例
file_paths = {
    'person1': 'data/feature/new_feature/features_with_labels_prerna.csv',
    'person2': 'data/feature/new_feature/features_with_labels_umang.csv',
    'person3': 'data/feature/new_feature/features_with_labels_yutaro.csv',
    'person4': 'data/feature/new_feature/features_with_labels_aoi.csv',
    'person5': 'data/feature/new_feature/features_with_labels_nabe.csv'
}

debug_head_banging_sequences(file_paths)


/content/drive/MyDrive/yolov7/Experiment
LabelEncoder classes: ['Attacking' 'Biting' 'Eating snacks' 'Head banging' 'Sitting quietly'
 'Throwing things' 'Using phone' 'Walking']
File: person1
Number of 'Head banging' frames: 414
Number of sequences created: 4066
File: person2
Number of 'Head banging' frames: 392
Number of sequences created: 4241
File: person3
Number of 'Head banging' frames: 262
Number of sequences created: 2505
File: person4
Number of 'Head banging' frames: 254
Number of sequences created: 2567
File: person5
Number of 'Head banging' frames: 262
Number of sequences created: 2634
All checks completed.
